In [15]:
library(tidyverse)
library(data.table)
source("../0_support-files//theme_CRP-MISC.R")

rds_list <- readRDS("../1_sample-data/DNAxRNA_decon.rds")
cfDNA_df_s = rds_list$cfdna
cfRNA_df_s = rds_list$cfrna

### non-overlap between cfrna_used and cfdna_used is due to longitudinal cfDNA samples not used in fig 4

In [3]:

##------------------------------------
## COMPARISONS
##------------------------------------
    comps_cfrna <- c("B.cell", "Mature.conventional.dendritic.cell", "Erythrocyte.erythroid.progenitor", "Kidney.epithelial.cell", "Intrahepatic.cholangiocyte",
                     "Respiratory.ciliated.cell", "Respiratory.secretory.cell", "Macrophage", "Monocyte", "neutrophil","NK.Cell",
                     "Pancreatic.Stellate.cell", "Pancreatic.alpha.beta.cell",
                     "T.Cell")
    comps_cfdna <- c("bcell", "dendritic", "erythroblast", "kidney", "liver", 
                     "lung", "lung", "macrophage", 'monocyte','neutrophil','nkcell', 
                     "pancreas", "pancreas",
                     "tcell")

    outputs <- list()

    for (i in 1:length(comps_cfrna)){

        cfrna_var = comps_cfrna[i]
        cfdna_var = comps_cfdna[i]

        stat <- suppressWarnings(cor.test(log((cfDNA_df_s[[cfdna_var]] * cfDNA_df_s$scaled_cfDNA)+.01),
                                          log(cfRNA_df_s[[cfrna_var]]+.01),
                                          method=c("pearson")))

        outputs[[paste0(cfrna_var,"_",cfdna_var)]] <- c("rna"=cfrna_var,"dna"=cfdna_var,"pval"=stat$p.value,"r"=stat$estimate)
    }

    output_df <- data.frame(do.call("rbind",outputs))
    
    output_df$adj.p <- p.adjust(output_df$pval, method = "BH")

    output_df %>% filter(r.cor > 0) %>% arrange(adj.p) %>% head(20)  

,rna,dna,pval,r.cor,adj.p
,<chr>,<chr>,<chr>,<chr>,<dbl>
B.cell_bcell,B.cell,bcell,0.0134192767761288,0.387791577240063,0.09393494
Erythrocyte.erythroid.progenitor_erythroblast,Erythrocyte.erythroid.progenitor,erythroblast,0.0122839244673481,0.392323533071603,0.09393494
Intrahepatic.cholangiocyte_liver,Intrahepatic.cholangiocyte,liver,0.0232227725494146,0.35823523771058,0.10837294
Kidney.epithelial.cell_kidney,Kidney.epithelial.cell,kidney,0.337227618005921,0.155745238290783,0.58015060
Respiratory.ciliated.cell_lung,Respiratory.ciliated.cell,lung,0.28528598322645,0.173158391404502,0.58015060
T.Cell_tcell,T.Cell,tcell,0.540942139839495,0.0995861595142219,0.63109916
Pancreatic.alpha.beta.cell_pancreas,Pancreatic.alpha.beta.cell,pancreas,0.764858479197887,0.0488126045689578,0.82369375
NK.Cell_nkcell,NK.Cell,nkcell,0.875235451152159,0.0256345785184772,0.87523545


In [10]:
make_bxplot <- function(cfDNA_df_s, cfRNA_df_s,
                        CFRNA_CELLTYPE, CFDNA_CELLTYPE){
    
    plot_df <- data.frame(
                        cfdna = cfDNA_df_s[[CFDNA_CELLTYPE]],
                      cfrna = cfRNA_df_s[[CFRNA_CELLTYPE]],
                     Diagnosis = cfDNA_df_s$Diagnosis)

    bxplt <- plot_df %>%
        ggplot(aes(x=cfrna, y = cfdna)) + 
        geom_point(size = 0.5) +
        theme_prevail()+
        theme(title = element_text(size = 7.5))+

        labs(title = paste0("cfrna: ",CFRNA_CELLTYPE,"\ncfdna: ",CFDNA_CELLTYPE),
            x = paste0(CFRNA_CELLTYPE,"\ncfRNA"),
            y = paste0(CFDNA_CELLTYPE,"\ncfDNA")
            )+
        theme(panel.grid.minor = element_blank(), title = element_blank())
}

In [11]:
WIDTH = 1.75
HEIGHT = 1.5

##------------------------------------

CFRNA_CELLTYPE = "Erythrocyte.erythroid.progenitor"
CFDNA_CELLTYPE = "erythroblast"

XLIM = .42
YLIM = .54

pdf(file=paste0("plots/SupPanelE_",paste(CFRNA_CELLTYPE,CFDNA_CELLTYPE,sep="_"),".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- make_bxplot(cfDNA_df_s, cfRNA_df_s,
                    CFRNA_CELLTYPE, CFDNA_CELLTYPE) +
    scale_x_continuous(breaks = c(0, XLIM/3, (XLIM/3)*2 , XLIM))+
    scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM))+
    coord_cartesian(ylim = c(0,YLIM*1.05),xlim = c(0,XLIM*1.05))

print(bxplt)

dev.off()

##------------------------------------

CFRNA_CELLTYPE = "B.cell"
CFDNA_CELLTYPE = "bcell"

XLIM = .06
YLIM = .3

pdf(file=paste0("plots/SupPanelE_",paste(CFRNA_CELLTYPE,CFDNA_CELLTYPE,sep="_"),".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- make_bxplot(cfDNA_df_s, cfRNA_df_s,
                    CFRNA_CELLTYPE, CFDNA_CELLTYPE) +
    scale_x_continuous(breaks = c(0, XLIM/3, (XLIM/3)*2 , XLIM))+
    scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM))+
    coord_cartesian(ylim = c(0,YLIM*1.05),xlim = c(0,XLIM*1.05))



print(bxplt)

dev.off()

##------------------------------------

CFRNA_CELLTYPE = "Intrahepatic.cholangiocyte"
CFDNA_CELLTYPE = "liver"

YLIM = .54
XLIM = .018

pdf(file=paste0("plots/SupPanelE_",paste(CFRNA_CELLTYPE,CFDNA_CELLTYPE,sep="_"),".pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

bxplt <- make_bxplot(cfDNA_df_s, cfRNA_df_s,
                    CFRNA_CELLTYPE, CFDNA_CELLTYPE) +
    scale_x_continuous(breaks = c(0, XLIM/3, (XLIM/3)*2 , XLIM))+
    scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM))+
    coord_cartesian(ylim = c(0,YLIM*1.05),xlim = c(0,XLIM*1.05))



print(bxplt)

dev.off()

png 
  2

png 
  2

png 
  2